In [1]:
import pymorphy2
import pandas as pd
from tqdm import tqdm
import string
import numpy as np

In [2]:
dataset = pd.read_csv('../dataset_2.csv')

In [3]:
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/appuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
tqdm.pandas()


class Searcher():
    def __init__(self, data, use_lemmatization, b=0.75, k=2, texts=None, inv_index=None):
        self.morph = pymorphy2.MorphAnalyzer()
        self.punctuation = string.punctuation + '…' + '«' + '»' + '—'
        self.stop_words = stopwords.words('russian')
        self.use_lemmatization = use_lemmatization

        self.k = k
        self.b = b

        self.data = data # исходные данные

        if texts is None:
            print('Preprocessing texts...')
            self.texts = data.text.progress_apply(self.preprocess_text) # очищенные тексты в нормальной форме
        else:
            self.texts = texts

        if inv_index is None:
            print('Counting inverted index...')

            inv_index = {}

            for doc_id, doc in enumerate(tqdm(self.texts)):
                tokens = doc.split()
                for token in tokens:
                    if token in inv_index:
                        inv_index[token].add(doc_id)
                    else:
                        inv_index[token] = set([doc_id])
        
        self.inv_index = inv_index

    def preprocess_text(self, text):
        tokens = str(text).lower().translate(str.maketrans('', '', self.punctuation)).split()
        res = ''

        if self.use_lemmatization:
            for token in tokens:
                p = self.morph.parse(token)[0].normal_form
                if p not in self.stop_words:
                    res += ' ' + p
        else:
            for token in tokens:
                if token not in self.stop_words:
                    res += ' ' + token
        
        return res


    def idf(self, terms, documents, eps=-0.01):
        idfs = np.asarray([len(documents) / (len(self.inv_index[term]) + eps) for term in terms])
        return np.log(idfs)


    def tf(self, terms, doc_ids):
        # можно посплитить текст в init и не сплитить каждый раз
        def count(text):
            freqs = dict.fromkeys(terms, 0)
            text = text.split() 
            for term in text:
                if term in freqs.keys():
                    freqs[term] += 1
            
            return np.asarray([term_freq[1] for term_freq in freqs.items()]) / len(text)

        docs = self.texts[doc_ids]

        return np.stack(docs.apply(count).values)
    
    def tf_idf(self, terms, documents):
        tf_scores = self.tf(terms, documents)
        idf_scores = self.idf(terms, documents)

        scores = tf_scores * idf_scores
        return scores

    def bm25(self, terms, documents):
        
        tf_scores = self.tf(terms, documents)
        idf_scores = self.idf(terms, documents)
        lens = self.texts[documents].apply(lambda x: len(x.split())).to_numpy()

        scores = np.zeros(lens.shape)
        for i, term in enumerate(terms):
            a = (tf_scores[:, i] * (self.k + 1))
            b = (tf_scores[:, i] + self.k * (1 - self.b + self.b * lens / lens.mean()))
            c = idf_scores[i]
            scores += a / b * c

        return scores

    def search(self, query):
        terms = self.preprocess_text(query).split()

        global_ids = set()
        for term in terms:
            global_ids.update(self.inv_index[term])

        global_ids = np.asarray(list(global_ids))
        scores = self.bm25(terms, global_ids)#.sum(axis=1)

        local_ids = np.argsort(scores, axis=0).squeeze() # сортируем документы по возрастанию tf-idf
        

        return self.data.iloc[global_ids[local_ids][-10:]]

        
        

In [8]:
s = Searcher(dataset, use_lemmatization=False)

Preprocessing texts...


100%|██████████| 10385/10385 [00:20<00:00, 505.43it/s] 


Counting inverted index...


100%|██████████| 10385/10385 [00:09<00:00, 1075.92it/s]


In [9]:
texts = s.texts

In [237]:
s.search('tomb raider')

,Unnamed: 0,title,link,text
5210,5210,Всегда чистый,/wiki/%D0%92%D1%81%D0%B5%D0%B3%D0%B4%D0%B0_%D1...,129. Второй Закон Моды Костюмы всех персонаже...
5080,5080,Рыба-кусака,/wiki/%D0%A0%D1%8B%D0%B1%D0%B0-%D0%BA%D1%83%D1...,Рыба-кусака — специфический вид противников в ...
5205,5205,Вид от сапога,/wiki/%D0%92%D0%B8%D0%B4_%D0%BE%D1%82_%D1%81%D...,"Вид от сапога — стилистический приём, специфич..."
5158,5158,Дай покататься!,/wiki/%D0%94%D0%B0%D0%B9_%D0%BF%D0%BE%D0%BA%D0...,"Популярный элемент в шутерах, когда главному г..."
8469,8469,Жанровая слепота,/wiki/%D0%96%D0%B0%D0%BD%D1%80%D0%BE%D0%B2%D0%...,xxx: что меня больше всего убивает в фильмах п...
8439,8439,В каждой бочке затычка,/wiki/%D0%92_%D0%BA%D0%B0%D0%B6%D0%B4%D0%BE%D0...,Этот второстепенный персонаж (не протагонист и...
5235,5235,Воздух — это земля,/wiki/%D0%92%D0%BE%D0%B7%D0%B4%D1%83%D1%85_%E2...,"Объединяющий троп для ряда ситуаций, когда пер..."
6405,6405,Tomb Raider,/wiki/Tomb_Raider,Tomb Raider — видеоигровая серия экшен-платфор...
5185,5185,Прыжковая головоломка,/wiki/%D0%9F%D1%80%D1%8B%D0%B6%D0%BA%D0%BE%D0%...,Прыжковая головоломка — это та же игра в класс...
3063,3063,Лара Крофт,/wiki/%D0%9B%D0%B0%D1%80%D0%B0_%D0%9A%D1%80%D0...,"Лара Крофт (aka Гробокопательница, Склепограби..."


In [238]:
s.search('the elder scrolls')

,Unnamed: 0,title,link,text
5071,5071,Единственный человек в цеху,/wiki/%D0%95%D0%B4%D0%B8%D0%BD%D1%81%D1%82%D0%...,"Ситуация в RPG-играх, когда, если в каком-либо..."
4876,4876,Васян,/wiki/%D0%92%D0%B0%D1%81%D1%8F%D0%BD,Васян — в контексте видеоигр и ПО так называют...
651,651,Jvk1166z.esp,/wiki/Jvk1166z.esp,Jvk1166z.esp (иногда Jvk.esp) — мощная крипипа...
5056,5056,Sandbox,/wiki/Sandbox,Песочница (англ. Sandbox) — тип компьютерной и...
5139,5139,Фальшивый босс,/wiki/%D0%A4%D0%B0%D0%BB%D1%8C%D1%88%D0%B8%D0%...,Вы дошли до босса и вынесли его с полпинка. А ...
564,564,Stay awhile and listen,/wiki/Stay_awhile_and_listen,Stay awhile and listen (Постой и послушай) — л...
5140,5140,Финальный босс,/wiki/%D0%A4%D0%B8%D0%BD%D0%B0%D0%BB%D1%8C%D0%...,"Именно то, что написано на упаковке. Последний..."
3074,3074,Довакин,/wiki/%D0%94%D0%BE%D0%B2%D0%B0%D0%BA%D0%B8%D0%BD,Довакин — главный герой видеоигры The Elder Sc...
5013,5013,Action-RPG,/wiki/Action-RPG,"Action-RPG (экшн-ролевая игра) — ролевая игра,..."
562,562,"Или живите дальше в проклятом мире, который са...",/wiki/%D0%98%D0%BB%D0%B8_%D0%B6%D0%B8%D0%B2%D0...,Со смертью этого персонажа нить вашей судьбы о...


In [239]:
# с лемматизацией
s.search('прострелили колено')

,Unnamed: 0,title,link,text
2904,2904,Brick Game,/wiki/Brick_Game,Brick Game или более привычно Тетрис — портати...
8572,8572,Поцеловать ногу,/wiki/%D0%9F%D0%BE%D1%86%D0%B5%D0%BB%D0%BE%D0%...,"Поцеловать ступню — это ещё более радикально, ..."
8712,8712,Собачье послушание,/wiki/%D0%A1%D0%BE%D0%B1%D0%B0%D1%87%D1%8C%D0%...,Люди холопского звания Сущие псы иногда: Чем т...
8513,8513,Минувшие приключения,/wiki/%D0%9C%D0%B8%D0%BD%D1%83%D0%B2%D1%88%D0%...,Когда-то и меня вела дорога приключений… А пот...
5586,5586,Приветствия Лепры,/wiki/%D0%9F%D1%80%D0%B8%D0%B2%D0%B5%D1%82%D1%...,Случайно показываемое приветствие на главной с...
5254,5254,Рэгдолл,/wiki/%D0%A0%D1%8D%D0%B3%D0%B4%D0%BE%D0%BB%D0%BB,ВкратцеПерсонаж падает не запрограммированной ...
8132,8132,144 тысячи,/wiki/144_%D1%82%D1%8B%D1%81%D1%8F%D1%87%D0%B8,"144 тысячи — священное число, количество людей..."
6341,6341,Бойцовский клуб,/wiki/%D0%91%D0%BE%D0%B9%D1%86%D0%BE%D0%B2%D1%...,Не упоминать о бойцовском клубе.— Первое (и вт...
3502,3502,Выстрелить себе в ногу,/wiki/%D0%92%D1%8B%D1%81%D1%82%D1%80%D0%B5%D0%...,"Смело, товарищи, в ногу! Духом окрепнем в борь..."
5598,5598,Прострели коленку,/wiki/%D0%9F%D1%80%D0%BE%D1%81%D1%82%D1%80%D0%...,1: Думаю что будет неожиданнее? Прийти на рабо...


In [12]:
# без лемматизации
s.search('прострелили колено')

,Unnamed: 0,title,link,text
9238,9238,Положил я Лене писю на колени,/wiki/%D0%9F%D0%BE%D0%BB%D0%BE%D0%B6%D0%B8%D0%...,Положил я Лене писю на колени У меня на Лену ч...
7324,7324,Павел Сенько,/wiki/%D0%9F%D0%B0%D0%B2%D0%B5%D0%BB_%D0%A1%D0...,"Павел Сенько (Pavel Senko, senko) — американск..."
7859,7859,Хватание за ляжку,/wiki/%D0%A5%D0%B2%D0%B0%D1%82%D0%B0%D0%BD%D0%...,Дева тешит до известного предела — дальше лок...
3828,3828,Герман Геринг,/wiki/%D0%93%D0%B5%D1%80%D0%BC%D0%B0%D0%BD_%D0...,"Народ, вне зависимости от того, наделен ли он ..."
1234,1234,2channel,/wiki/2channel,2channel (moon. 2ちゃんねる) — крупнейший в мире яп...
493,493,Skyrim,/wiki/Skyrim,"Душа, душа на каждом шагу, а в локации с забро..."
8520,8520,Мультикилл,/wiki/%D0%9C%D1%83%D0%BB%D1%8C%D1%82%D0%B8%D0%...,ВкратцеУбить нескольких врагов одним действием...
10042,10042,Борис Акунин,/wiki/%D0%91%D0%BE%D1%80%D0%B8%D1%81_%D0%90%D0...,"Если спрошено будет, чьи произведения хуже: До..."
8513,8513,Минувшие приключения,/wiki/%D0%9C%D0%B8%D0%BD%D1%83%D0%B2%D1%88%D0%...,Когда-то и меня вела дорога приключений… А пот...
5598,5598,Прострели коленку,/wiki/%D0%9F%D1%80%D0%BE%D1%81%D1%82%D1%80%D0%...,1: Думаю что будет неожиданнее? Прийти на рабо...


In [240]:
s.search('капибара')

,Unnamed: 0,title,link,text
8969,8969,Стас Давыдов,/wiki/%D0%A1%D1%82%D0%B0%D1%81_%D0%94%D0%B0%D0...,"ПРИВЕТ, Я СТАС ДАВЫДОВ! — Сабж о себе После..."
4672,4672,Пикабу,/wiki/%D0%9F%D0%B8%D0%BA%D0%B0%D0%B1%D1%83,Пикабу — ставший в последнее время довольно по...
361,361,Животные за компьютером,/wiki/%D0%96%D0%B8%D0%B2%D0%BE%D1%82%D0%BD%D1%...,Животные за компьютером — распространённая кат...


In [13]:
# без лемматизации
s.search('животные за компьютером')

,Unnamed: 0,title,link,text
621,621,Ебланы (FatCat),/wiki/%D0%95%D0%B1%D0%BB%D0%B0%D0%BD%D1%8B_(Fa...,Ебланы (FatCat) — распространённый мем среди с...
5276,5276,PvE,/wiki/PvE,Player versus Environment (Игрок против окруже...
9745,9745,Ретрит,/wiki/%D0%A0%D0%B5%D1%82%D1%80%D0%B8%D1%82,Ретрит (от англ. retreat) — религиозная практи...
3120,3120,Гейм-мастер,/wiki/%D0%93%D0%B5%D0%B9%D0%BC-%D0%BC%D0%B0%D1...,"Гейм-мастер (Game Master, GM/ГМ — прямой перев..."
323,323,Права животных,/wiki/%D0%9F%D1%80%D0%B0%D0%B2%D0%B0_%D0%B6%D0...,"Права животных, «освобождение животных» (англ...."
1703,1703,Автозамена на прокси,/wiki/%D0%90%D0%B2%D1%82%D0%BE%D0%B7%D0%B0%D0%...,"Автозамена на прокси — событие, когда определё..."
3025,3025,PvP,/wiki/PvP,"PvP (player vs player, игрок против игрока) — ..."
5440,5440,Жизнь,/wiki/%D0%96%D0%B8%D0%B7%D0%BD%D1%8C,"Остаточное свечение люминофора уснувшей души, ..."
2192,2192,Да кто такой этот ваш,/wiki/%D0%94%D0%B0_%D0%BA%D1%82%D0%BE_%D1%82%D...,"Да кто такой этот ваш — интернет-мем, довольна..."
361,361,Животные за компьютером,/wiki/%D0%96%D0%B8%D0%B2%D0%BE%D1%82%D0%BD%D1%...,Животные за компьютером — распространённая кат...


In [241]:
# с лемматизацией
s.search('животные за компьютером')

,Unnamed: 0,title,link,text
71,71,Слон,/wiki/%D0%A1%D0%BB%D0%BE%D0%BD,"Слон — весьма большое животное, отличающееся д..."
1603,1603,IP-адрес,/wiki/IP-%D0%B0%D0%B4%D1%80%D0%B5%D1%81,"IP-адрес (IP address, айпи) — идентификатор ко..."
6947,6947,Нейросети и актёры,/wiki/%D0%9D%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%...,Нейросети и актёры — гипотетическая замена жив...
251,251,Кормление бездомных животных,/wiki/%D0%9A%D0%BE%D1%80%D0%BC%D0%BB%D0%B5%D0%...,Кормление бездомных животных — процесс организ...
399,399,Краска для домашних животных,/wiki/%D0%9A%D1%80%D0%B0%D1%81%D0%BA%D0%B0_%D0...,Краска для домашних животных — очередная довол...
2283,2283,Бог создаёт животных,/wiki/%D0%91%D0%BE%D0%B3_%D1%81%D0%BE%D0%B7%D0...,Бог создаёт животных — популярная тема для мем...
2266,2266,"Ура! Я умнее, чем компьютер!",/wiki/%D0%A3%D1%80%D0%B0!_%D0%AF_%D1%83%D0%BC%...,"Ха-ха-ха-ха-ха, я умнее тебя! У меня памяти 16..."
323,323,Права животных,/wiki/%D0%9F%D1%80%D0%B0%D0%B2%D0%B0_%D0%B6%D0...,"Права животных, «освобождение животных» (англ...."
407,407,Животных не кормить,/wiki/%D0%96%D0%B8%D0%B2%D0%BE%D1%82%D0%BD%D1%...,"Животных не кормить — крылатая фраза, распрост..."
361,361,Животные за компьютером,/wiki/%D0%96%D0%B8%D0%B2%D0%BE%D1%82%D0%BD%D1%...,Животные за компьютером — распространённая кат...


In [242]:
s.search('лошади')

,Unnamed: 0,title,link,text
5980,5980,Не нужен,/wiki/%D0%9D%D0%B5_%D0%BD%D1%83%D0%B6%D0%B5%D0%BD,"Не нужен (не нужно и т. д.) — фраза, широко ис..."
260,260,Зверолюди,/wiki/%D0%97%D0%B2%D0%B5%D1%80%D0%BE%D0%BB%D1%...,Зверолюди — гибриды человека и некоего животно...
2285,2285,Догпилл,/wiki/%D0%94%D0%BE%D0%B3%D0%BF%D0%B8%D0%BB%D0%BB,Догпилл (Dogpill) — название для популярного н...
413,413,Конь,/wiki/%D0%9A%D0%BE%D0%BD%D1%8C,Это страница неоднозначности. Ниже можно прочи...
5717,5717,Ты бы и собаку,/wiki/%D0%A2%D1%8B_%D0%B1%D1%8B_%D0%B8_%D1%81%...,Ты бы и собаку… — распространенное на Дваче и ...
1651,1651,Вархаммер против пони,/wiki/%D0%92%D0%B0%D1%80%D1%85%D0%B0%D0%BC%D0%...,Вархаммер против пони — известный мем на Joyre...
421,421,Осёл,/wiki/%D0%9E%D1%81%D1%91%D0%BB,Осёл — весьма выносливое домашнее животное лош...
2286,2286,Sometimes you just know,/wiki/Sometimes_you_just_know,Sometimes you just know — распространённый анг...
1995,1995,Ржание коня,/wiki/%D0%A0%D0%B6%D0%B0%D0%BD%D0%B8%D0%B5_%D0...,"Ржание коня — распространенный звук, который и..."
6976,6976,Зорот,/wiki/%D0%97%D0%BE%D1%80%D0%BE%D1%82,Зорот (Xoroth) — мир из вселенной Warcraft. Эт...


In [245]:
dataset.iloc[6976].text

'Зорот (Xoroth) — мир из вселенной Warcraft. Эта планета принадлежит Пылающему Легиону, в частности, натрезимам, и является родиной коней погибели, которых чернокнижники используют в качестве верховых лошадей.'

In [250]:
s.search('быдло')

,Unnamed: 0,title,link,text
1080,1080,Позязя,/wiki/%D0%9F%D0%BE%D0%B7%D1%8F%D0%B7%D1%8F,Позязя — искаженное «пожалуйста». Используется...
1322,1322,Школота,/wiki/%D0%A8%D0%BA%D0%BE%D0%BB%D0%BE%D1%82%D0%B0,"Школота (школиё, школий, личинка человека, они..."
1089,1089,Ихний,/wiki/%D0%98%D1%85%D0%BD%D0%B8%D0%B9,Ихний (название в периодической системе Ih; та...
1088,1088,Нету,/wiki/%D0%9D%D0%B5%D1%82%D1%83,"Нету — популярное среди быдла слово, которого ..."
1981,1981,"Блядство, разврат и наркотики",/wiki/%D0%91%D0%BB%D1%8F%D0%B4%D1%81%D1%82%D0%...,"Блядство, разврат и наркотики (дворянский Безо..."
5738,5738,Чемпионат Европы,/wiki/%D0%A7%D0%B5%D0%BC%D0%BF%D0%B8%D0%BE%D0%...,"Самарские битарды состоят из БЫДЛО, БЫДЛО, БЫД..."
5714,5714,Ночной Двач,/wiki/%D0%9D%D0%BE%D1%87%D0%BD%D0%BE%D0%B9_%D0...,"Жарское, раннее июньское утроНа дваче сидят дв..."
4006,4006,Небыдло,/wiki/%D0%9D%D0%B5%D0%B1%D1%8B%D0%B4%D0%BB%D0%BE,"Небыдло — особый подтип быдла, который себя та..."
1347,1347,Бугагашечки,/wiki/%D0%91%D1%83%D0%B3%D0%B0%D0%B3%D0%B0%D1%...,В небезызвестном комедийном сериале Теория бол...
3971,3971,Юморное:Теории о быдле,/wiki/%D0%AE%D0%BC%D0%BE%D1%80%D0%BD%D0%BE%D0%...,Гомогенная модель быдла основывается на предпо...


In [253]:
len(dataset.iloc[3971].text.split())

720

In [256]:
s = Searcher(dataset, texts=texts, b=0.25)

Counting inverted index...


100%|██████████| 10385/10385 [00:12<00:00, 863.43it/s] 


In [248]:
s.search('быдло')

,Unnamed: 0,title,link,text
1089,1089,Ихний,/wiki/%D0%98%D1%85%D0%BD%D0%B8%D0%B9,Ихний (название в периодической системе Ih; та...
1322,1322,Школота,/wiki/%D0%A8%D0%BA%D0%BE%D0%BB%D0%BE%D1%82%D0%B0,"Школота (школиё, школий, личинка человека, они..."
5738,5738,Чемпионат Европы,/wiki/%D0%A7%D0%B5%D0%BC%D0%BF%D0%B8%D0%BE%D0%...,"Самарские битарды состоят из БЫДЛО, БЫДЛО, БЫД..."
1347,1347,Бугагашечки,/wiki/%D0%91%D1%83%D0%B3%D0%B0%D0%B3%D0%B0%D1%...,В небезызвестном комедийном сериале Теория бол...
5714,5714,Ночной Двач,/wiki/%D0%9D%D0%BE%D1%87%D0%BD%D0%BE%D0%B9_%D0...,"Жарское, раннее июньское утроНа дваче сидят дв..."
4647,4647,Быдло/Вариант из Луркоморья,/wiki/%D0%91%D1%8B%D0%B4%D0%BB%D0%BE/%D0%92%D0...,Sapere aude— Латинское изречение Немногие дейс...
4006,4006,Небыдло,/wiki/%D0%9D%D0%B5%D0%B1%D1%8B%D0%B4%D0%BB%D0%BE,"Небыдло — особый подтип быдла, который себя та..."
9,9,Быдло,/wiki/%D0%91%D1%8B%D0%B4%D0%BB%D0%BE,Будь проще — шути больше— Заветы всего быдла ...
4648,4648,Старое обсуждение:Быдло,/wiki/%D0%A1%D1%82%D0%B0%D1%80%D0%BE%D0%B5_%D0...,"Внимание! Это старая дискуссия, которая некогд..."
3971,3971,Юморное:Теории о быдле,/wiki/%D0%AE%D0%BC%D0%BE%D1%80%D0%BD%D0%BE%D0%...,Гомогенная модель быдла основывается на предпо...


In [254]:
len(dataset.iloc[9].text.split())

3685

In [257]:
s.search('лошади')

,Unnamed: 0,title,link,text
2613,2613,Александр Невзоров,/wiki/%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%...,"Никогда не думала, что в человеке может быть с..."
2285,2285,Догпилл,/wiki/%D0%94%D0%BE%D0%B3%D0%BF%D0%B8%D0%BB%D0%BB,Догпилл (Dogpill) — название для популярного н...
413,413,Конь,/wiki/%D0%9A%D0%BE%D0%BD%D1%8C,Это страница неоднозначности. Ниже можно прочи...
5717,5717,Ты бы и собаку,/wiki/%D0%A2%D1%8B_%D0%B1%D1%8B_%D0%B8_%D1%81%...,Ты бы и собаку… — распространенное на Дваче и ...
67,67,Лошадь,/wiki/%D0%9B%D0%BE%D1%88%D0%B0%D0%B4%D1%8C,"— …Любишь лошадей? — Хм, — задумался я, прикид..."
1651,1651,Вархаммер против пони,/wiki/%D0%92%D0%B0%D1%80%D1%85%D0%B0%D0%BC%D0%...,Вархаммер против пони — известный мем на Joyre...
421,421,Осёл,/wiki/%D0%9E%D1%81%D1%91%D0%BB,Осёл — весьма выносливое домашнее животное лош...
2286,2286,Sometimes you just know,/wiki/Sometimes_you_just_know,Sometimes you just know — распространённый анг...
1995,1995,Ржание коня,/wiki/%D0%A0%D0%B6%D0%B0%D0%BD%D0%B8%D0%B5_%D0...,"Ржание коня — распространенный звук, который и..."
6976,6976,Зорот,/wiki/%D0%97%D0%BE%D1%80%D0%BE%D1%82,Зорот (Xoroth) — мир из вселенной Warcraft. Эт...


In [265]:
s = Searcher(dataset, texts=texts, b=0.1)

Counting inverted index...


100%|██████████| 10385/10385 [00:10<00:00, 956.77it/s] 


In [266]:
s.search('лошади')

,Unnamed: 0,title,link,text
2285,2285,Догпилл,/wiki/%D0%94%D0%BE%D0%B3%D0%BF%D0%B8%D0%BB%D0%BB,Догпилл (Dogpill) — название для популярного н...
413,413,Конь,/wiki/%D0%9A%D0%BE%D0%BD%D1%8C,Это страница неоднозначности. Ниже можно прочи...
2613,2613,Александр Невзоров,/wiki/%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%...,"Никогда не думала, что в человеке может быть с..."
5717,5717,Ты бы и собаку,/wiki/%D0%A2%D1%8B_%D0%B1%D1%8B_%D0%B8_%D1%81%...,Ты бы и собаку… — распространенное на Дваче и ...
1651,1651,Вархаммер против пони,/wiki/%D0%92%D0%B0%D1%80%D1%85%D0%B0%D0%BC%D0%...,Вархаммер против пони — известный мем на Joyre...
67,67,Лошадь,/wiki/%D0%9B%D0%BE%D1%88%D0%B0%D0%B4%D1%8C,"— …Любишь лошадей? — Хм, — задумался я, прикид..."
421,421,Осёл,/wiki/%D0%9E%D1%81%D1%91%D0%BB,Осёл — весьма выносливое домашнее животное лош...
2286,2286,Sometimes you just know,/wiki/Sometimes_you_just_know,Sometimes you just know — распространённый анг...
1995,1995,Ржание коня,/wiki/%D0%A0%D0%B6%D0%B0%D0%BD%D0%B8%D0%B5_%D0...,"Ржание коня — распространенный звук, который и..."
6976,6976,Зорот,/wiki/%D0%97%D0%BE%D1%80%D0%BE%D1%82,Зорот (Xoroth) — мир из вселенной Warcraft. Эт...


In [21]:
s = Searcher(dataset, use_lemmatization=True)

Preprocessing texts...


100%|██████████| 10385/10385 [04:29<00:00, 38.54it/s] 


Counting inverted index...


100%|██████████| 10385/10385 [00:15<00:00, 658.63it/s] 


In [16]:
# без лемматизации
s.search('конституция российской федерации')

,Unnamed: 0,title,link,text
4542,4542,КГ/АМ,/wiki/%D0%9A%D0%93/%D0%90%D0%9C,КГ/АМ — сокращенная форма от фразы «креатифф г...
4749,4749,Соотечественник,/wiki/%D0%A1%D0%BE%D0%BE%D1%82%D0%B5%D1%87%D0%...,"Соотечественник — человек, проживающий за пред..."
3808,3808,Игорь Конашенков,/wiki/%D0%98%D0%B3%D0%BE%D1%80%D1%8C_%D0%9A%D0...,Игорь Евгеньевич Конашенков — российский военн...
3248,3248,Эквадор,/wiki/%D0%AD%D0%BA%D0%B2%D0%B0%D0%B4%D0%BE%D1%80,"Эквадор — страна в Южной Америке, граничит с К..."
9439,9439,Аннексия Новороссийской губернии Украиной,/wiki/%D0%90%D0%BD%D0%BD%D0%B5%D0%BA%D1%81%D0%...,Аннексия Новороссийской губернии Украиной — не...
3348,3348,Новая Москва,/wiki/%D0%9D%D0%BE%D0%B2%D0%B0%D1%8F_%D0%9C%D0...,«Новая Москва» — юго-западный сектор Подмосков...
5602,5602,Лепропедия,/wiki/%D0%9B%D0%B5%D0%BF%D1%80%D0%BE%D0%BF%D0%...,"Логотип Лепропедии. Глагне Лепропедии Чатик, х..."
9815,9815,Всеволод Чаплин,/wiki/%D0%92%D1%81%D0%B5%D0%B2%D0%BE%D0%BB%D0%...,"Всеволод Чаплин — поп-звезда, профессиональный..."
9819,9819,Храм шаговой доступности,/wiki/%D0%A5%D1%80%D0%B0%D0%BC_%D1%88%D0%B0%D0...,Храм шаговой доступности — государственная бес...
2203,2203,Педерация,/wiki/%D0%9F%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%...,Педерация — распространённое искажение слова «...


In [23]:
# с лемматизацией
s.search('конституция российской федерации')

,Unnamed: 0,title,link,text
3248,3248,Эквадор,/wiki/%D0%AD%D0%BA%D0%B2%D0%B0%D0%B4%D0%BE%D1%80,"Эквадор — страна в Южной Америке, граничит с К..."
3808,3808,Игорь Конашенков,/wiki/%D0%98%D0%B3%D0%BE%D1%80%D1%8C_%D0%9A%D0...,Игорь Евгеньевич Конашенков — российский военн...
7633,7633,Русофобия русских американцев,/wiki/%D0%A0%D1%83%D1%81%D0%BE%D1%84%D0%BE%D0%...,Ненависть к России русских американцев — явлен...
3348,3348,Новая Москва,/wiki/%D0%9D%D0%BE%D0%B2%D0%B0%D1%8F_%D0%9C%D0...,«Новая Москва» — юго-западный сектор Подмосков...
9815,9815,Всеволод Чаплин,/wiki/%D0%92%D1%81%D0%B5%D0%B2%D0%BE%D0%BB%D0%...,"Всеволод Чаплин — поп-звезда, профессиональный..."
9819,9819,Храм шаговой доступности,/wiki/%D0%A5%D1%80%D0%B0%D0%BC_%D1%88%D0%B0%D0...,Храм шаговой доступности — государственная бес...
2238,2238,Котенька,/wiki/%D0%9A%D0%BE%D1%82%D0%B5%D0%BD%D1%8C%D0%...,"Котенька (Grafics Cat, Starecat, Круглоглазая ..."
9439,9439,Аннексия Новороссийской губернии Украиной,/wiki/%D0%90%D0%BD%D0%BD%D0%B5%D0%BA%D1%81%D0%...,Аннексия Новороссийской губернии Украиной — не...
2337,2337,Родитель №1 и родитель №2,/wiki/%D0%A0%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%...,Родитель № 1 и родитель № 2 — меметичная фраза...
2203,2203,Педерация,/wiki/%D0%9F%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%...,Педерация — распространённое искажение слова «...


In [17]:
# без лемматизации
s.search('война в украине')

,Unnamed: 0,title,link,text
3418,3418,Кривой Рог,/wiki/%D0%9A%D1%80%D0%B8%D0%B2%D0%BE%D0%B9_%D0...,Кривой Рог — город в центре Украины. Входит в ...
9494,9494,Война интеллекта с глупостью,/wiki/%D0%92%D0%BE%D0%B9%D0%BD%D0%B0_%D0%B8%D0...,Война интеллекта с глупостью — извечная война ...
7451,7451,Холмогорская Резьба,/wiki/%D0%A5%D0%BE%D0%BB%D0%BC%D0%BE%D0%B3%D0%...,Холмогорская Резьба — авторская программа росс...
4729,4729,Рашисты,/wiki/%D0%A0%D0%B0%D1%88%D0%B8%D1%81%D1%82%D1%8B,Рашисты — рашинские фашисты (российские боевик...
10308,10308,Нацизм на Украине,/wiki/%D0%9D%D0%B0%D1%86%D0%B8%D0%B7%D0%BC_%D0...,"Нацизм на Украине — важное явление, которое за..."
7667,7667,Миру мир,/wiki/%D0%9C%D0%B8%D1%80%D1%83_%D0%BC%D0%B8%D1%80,"Миру мир — известный лозунг, который использов..."
3416,3416,Полтава,/wiki/%D0%9F%D0%BE%D0%BB%D1%82%D0%B0%D0%B2%D0%B0,"Полтава — город на Восточной Украине, областно..."
10362,10362,Евгений Викторович Кошевой,/wiki/%D0%95%D0%B2%D0%B3%D0%B5%D0%BD%D0%B8%D0%...,Евгений Викторович Кошевой - лысый придурок и ...
2105,2105,"Сало уронили, героям сала!",/wiki/%D0%A1%D0%B0%D0%BB%D0%BE_%D1%83%D1%80%D0...,"Сало уронили, героям сало! (или просто сало ур..."
2830,2830,Андрей Сапунов,/wiki/%D0%90%D0%BD%D0%B4%D1%80%D0%B5%D0%B9_%D0...,Андрей Сапунов или Пан Сапунов — поехавший пок...


In [24]:
# с лемматизацией
s.search('война в украине')

,Unnamed: 0,title,link,text
7458,7458,Конец истории (Вершинин),/wiki/%D0%9A%D0%BE%D0%BD%D0%B5%D1%86_%D0%B8%D1...,Конец истории — заголовок для одного из постов...
2074,2074,Анти-Россия,/wiki/%D0%90%D0%BD%D1%82%D0%B8-%D0%A0%D0%BE%D1...,Анти-Россия — название для условного проекта п...
2830,2830,Андрей Сапунов,/wiki/%D0%90%D0%BD%D0%B4%D1%80%D0%B5%D0%B9_%D0...,Андрей Сапунов или Пан Сапунов — поехавший пок...
10314,10314,Украина це Европа,/wiki/%D0%A3%D0%BA%D1%80%D0%B0%D0%B8%D0%BD%D0%...,"Украина це Европа (укр. Украина это Европа, Ук..."
2083,2083,Их там нет,/wiki/%D0%98%D1%85_%D1%82%D0%B0%D0%BC_%D0%BD%D...,"Родина о тебе даже не узнает, сынок.— Армия ка..."
9429,9429,Украинский кризис,/wiki/%D0%A3%D0%BA%D1%80%D0%B0%D0%B8%D0%BD%D1%...,Украинский кризис — политический кризис с учас...
10383,10383,Флаг Украины,/wiki/%D0%A4%D0%BB%D0%B0%D0%B3_%D0%A3%D0%BA%D1...,"Флаг Украины — весьма специфический флаг, кото..."
10362,10362,Евгений Викторович Кошевой,/wiki/%D0%95%D0%B2%D0%B3%D0%B5%D0%BD%D0%B8%D0%...,Евгений Викторович Кошевой - лысый придурок и ...
2125,2125,Какие же хохлы дегенераты,/wiki/%D0%9A%D0%B0%D0%BA%D0%B8%D0%B5_%D0%B6%D0...,"Фраза, брошенная вскользь в одном из тредов на..."
4729,4729,Рашисты,/wiki/%D0%A0%D0%B0%D1%88%D0%B8%D1%81%D1%82%D1%8B,Рашисты — рашинские фашисты (российские боевик...


In [18]:
# без лемматизации
s.search('французская революция')

,Unnamed: 0,title,link,text
10197,10197,Звезда футбола,/wiki/%D0%97%D0%B2%D0%B5%D0%B7%D0%B4%D0%B0_%D1...,"Звезда футбола — футболист, который сделал сво..."
9008,9008,Nyan Cat,/wiki/Nyan_Cat,"Котэм нянь буй-буй будэ, сюлэмы керектэ.— Бура..."
1713,1713,Баяновая революция,/wiki/%D0%91%D0%B0%D1%8F%D0%BD%D0%BE%D0%B2%D0%...,"Баяновая революция («Бунд на Пикабу», «Первая ..."
4336,4336,Сего не буде,/wiki/%D0%A1%D0%B5%D0%B3%D0%BE_%D0%BD%D0%B5_%D...,Сего не буде — известная фраза из творчества Б...
8284,8284,Пейринг,/wiki/%D0%9F%D0%B5%D0%B9%D1%80%D0%B8%D0%BD%D0%B3,И любит кто кого — я знал…— Французская народн...
1541,1541,Рерайтинг,/wiki/%D0%A0%D0%B5%D1%80%D0%B0%D0%B9%D1%82%D0%...,Рерайтинг — один из основных методов генерации...
7613,7613,Мировая революция,/wiki/%D0%9C%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D1%...,"Мировая революция (Миговая геволюция, מהפכה עו..."
5312,5312,В морг,/wiki/%D0%92_%D0%BC%D0%BE%D1%80%D0%B3,"— Я случайно удалил win.com, что мне теперь де..."
9422,9422,Великая Октябрьская революция,/wiki/%D0%92%D0%B5%D0%BB%D0%B8%D0%BA%D0%B0%D1%...,Великая Октябрьская революция — сугубо коммуни...
7482,7482,Августовский путч,/wiki/%D0%90%D0%B2%D0%B3%D1%83%D1%81%D1%82%D0%...,Августовский путч (альт. Великая Августейшая А...


In [25]:
# с лемматизацией
s.search('французская революция')

,Unnamed: 0,title,link,text
5312,5312,В морг,/wiki/%D0%92_%D0%BC%D0%BE%D1%80%D0%B3,"— Я случайно удалил win.com, что мне теперь де..."
4336,4336,Сего не буде,/wiki/%D0%A1%D0%B5%D0%B3%D0%BE_%D0%BD%D0%B5_%D...,Сего не буде — известная фраза из творчества Б...
7506,7506,Феликс Дзержинский,/wiki/%D0%A4%D0%B5%D0%BB%D0%B8%D0%BA%D1%81_%D0...,"Это очень здорово, что мы помним об этом челов..."
2366,2366,Вы просто ненавидите всё русское,/wiki/%D0%92%D1%8B_%D0%BF%D1%80%D0%BE%D1%81%D1...,"Мы знаем, есть еще семейки, Где наше хают и бр..."
4918,4918,Редиска (жаргонизм),/wiki/%D0%A0%D0%B5%D0%B4%D0%B8%D1%81%D0%BA%D0%...,"«Редиска» — плохой человек, падла. Во время ..."
9422,9422,Великая Октябрьская революция,/wiki/%D0%92%D0%B5%D0%BB%D0%B8%D0%BA%D0%B0%D1%...,Великая Октябрьская революция — сугубо коммуни...
3867,3867,05.11.17,/wiki/05.11.17,"5.11.17 (05.11.2017, 5/11/2017, 05.11.17, 5.11..."
7482,7482,Августовский путч,/wiki/%D0%90%D0%B2%D0%B3%D1%83%D1%81%D1%82%D0%...,Августовский путч (альт. Великая Августейшая А...
7613,7613,Мировая революция,/wiki/%D0%9C%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D1%...,"Мировая революция (Миговая геволюция, מהפכה עו..."
964,964,Извините мой французский,/wiki/%D0%98%D0%B7%D0%B2%D0%B8%D0%BD%D0%B8%D1%...,Извиняюсь за свой французский (варианты: извин...


In [19]:
# без лемматизации
s.search('интерсекциональный феминизм третьей волны')

,Unnamed: 0,title,link,text
7614,7614,Красное смещение,/wiki/%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D0%BE%D0%...,"Красное смещение — инцидент, в ходе которого т..."
1761,1761,Лукоморье,/wiki/%D0%9B%D1%83%D0%BA%D0%BE%D0%BC%D0%BE%D1%...,Лукоморье — сказочное место из поэмы Пушкина. ...
7324,7324,Павел Сенько,/wiki/%D0%9F%D0%B0%D0%B2%D0%B5%D0%BB_%D0%A1%D0...,"Павел Сенько (Pavel Senko, senko) — американск..."
6806,6806,"Барби (фильм, 2023)",/wiki/%D0%91%D0%B0%D1%80%D0%B1%D0%B8_(%D1%84%D...,Барби (Barbie — от названия одноимённых кукол)...
6508,6508,Феминизм,/wiki/%D0%A4%D0%B5%D0%BC%D0%B8%D0%BD%D0%B8%D0%...,"Я задумался. Да, с одной стороны это можно был..."
4437,4437,Мужики vs бабы,/wiki/%D0%9C%D1%83%D0%B6%D0%B8%D0%BA%D0%B8_vs_...,"Мужики vs бабы — один из видов срачей, вытекаю..."
6848,6848,Добыча (фильм),/wiki/%D0%94%D0%BE%D0%B1%D1%8B%D1%87%D0%B0_(%D...,Добыча (Prey) — американский художественный фи...
1266,1266,Цунами,/wiki/%D0%A6%D1%83%D0%BD%D0%B0%D0%BC%D0%B8,Цунами — японская национальная периодическая к...
2776,2776,Pussy Riot,/wiki/Pussy_Riot,"Pussy Riot (бунт п##ды) — скандальная, известн..."
3650,3650,Кризис,/wiki/%D0%9A%D1%80%D0%B8%D0%B7%D0%B8%D1%81,"Большой пиздец не за горами, и волны кризисов ..."


In [26]:
# с лемматизацией
s.search('интерсекциональный феминизм третьей волны')

,Unnamed: 0,title,link,text
7883,7883,Культура изнасилования,/wiki/%D0%9A%D1%83%D0%BB%D1%8C%D1%82%D1%83%D1%...,Культура изнасилования — манипулятивный термин...
6848,6848,Добыча (фильм),/wiki/%D0%94%D0%BE%D0%B1%D1%8B%D1%87%D0%B0_(%D...,Добыча (Prey) — американский художественный фи...
9902,9902,Барбара Уолкер,/wiki/%D0%91%D0%B0%D1%80%D0%B1%D0%B0%D1%80%D0%...,Барбара Уолкер (Barbara Walker) — американская...
4938,4938,Сельдь,/wiki/%D0%A1%D0%B5%D0%BB%D1%8C%D0%B4%D1%8C,Сельдь (селедка) — распространенное на имиджбо...
1476,1476,Интернет-феминизм,/wiki/%D0%98%D0%BD%D1%82%D0%B5%D1%80%D0%BD%D0%...,Интернет-феминизм (или сетевой феминизм) — поп...
7249,7249,Kaeley Triller,/wiki/Kaeley_Triller,"Kaeley Triller (KaeleyT) — аккаунт в Твиттере,..."
1266,1266,Цунами,/wiki/%D0%A6%D1%83%D0%BD%D0%B0%D0%BC%D0%B8,Цунами — японская национальная периодическая к...
2776,2776,Pussy Riot,/wiki/Pussy_Riot,"Pussy Riot (бунт п##ды) — скандальная, известн..."
4746,4746,Мужчины-феминисты,/wiki/%D0%9C%D1%83%D0%B6%D1%87%D0%B8%D0%BD%D1%...,"Мужчины-феминисты — парадоксальное явление, до..."
3650,3650,Кризис,/wiki/%D0%9A%D1%80%D0%B8%D0%B7%D0%B8%D1%81,"Большой пиздец не за горами, и волны кризисов ..."


In [20]:
# без лемматизации
s.search('война судного дня')

,Unnamed: 0,title,link,text
5946,5946,Изенарелла,/wiki/%D0%98%D0%B7%D0%B5%D0%BD%D0%B0%D1%80%D0%...,"Изенарелла, также изенарела — это обозначение ..."
4886,4886,Военный эксперт,/wiki/%D0%92%D0%BE%D0%B5%D0%BD%D0%BD%D1%8B%D0%...,Военный эксперт — специалист по ведению военны...
9932,9932,Библейская цитата дня,/wiki/%D0%91%D0%B8%D0%B1%D0%BB%D0%B5%D0%B9%D1%...,Библейская цитата дня (slovobojie) — сообществ...
9223,9223,War (песня),/wiki/War_(%D0%BF%D0%B5%D1%81%D0%BD%D1%8F),"War — американская песня, которую сочинили и и..."
9522,9522,Финно-корейская гипервойна,/wiki/%D0%A4%D0%B8%D0%BD%D0%BD%D0%BE-%D0%BA%D0...,Финно-корейская гипервойна (Finno-Korean Hyper...
10218,10218,AIRSOFT BALAN,/wiki/AIRSOFT_BALAN,AIRSOFT BALAN — хороший YouTube-канал про стра...
8963,8963,Бикини-Боттом,/wiki/%D0%91%D0%B8%D0%BA%D0%B8%D0%BD%D0%B8-%D0...,Бикини-Боттом — он же вымышленный город в муль...
6985,6985,Третья Война (WarCraft),/wiki/%D0%A2%D1%80%D0%B5%D1%82%D1%8C%D1%8F_%D0...,Третья Война (Third War) — война Альянса и Нов...
9494,9494,Война интеллекта с глупостью,/wiki/%D0%92%D0%BE%D0%B9%D0%BD%D0%B0_%D0%B8%D0...,Война интеллекта с глупостью — извечная война ...
4241,4241,Яков Кедми,/wiki/%D0%AF%D0%BA%D0%BE%D0%B2_%D0%9A%D0%B5%D0...,Яков Кедми — израильский государственный деяте...


In [27]:
# с лемматизацией
s.search('война судного дня')

,Unnamed: 0,title,link,text
7034,7034,Тёмный Портал,/wiki/%D0%A2%D1%91%D0%BC%D0%BD%D1%8B%D0%B9_%D0...,Темный Портал (Dark Portal) — портал между Дре...
9522,9522,Финно-корейская гипервойна,/wiki/%D0%A4%D0%B8%D0%BD%D0%BD%D0%BE-%D0%BA%D0...,Финно-корейская гипервойна (Finno-Korean Hyper...
6985,6985,Третья Война (WarCraft),/wiki/%D0%A2%D1%80%D0%B5%D1%82%D1%8C%D1%8F_%D0...,Третья Война (Third War) — война Альянса и Нов...
7014,7014,Огры (WarCraft),/wiki/%D0%9E%D0%B3%D1%80%D1%8B_(WarCraft),Огры — это большие создания. У них светлая кож...
7629,7629,Дни памяти и примирения,/wiki/%D0%94%D0%BD%D0%B8_%D0%BF%D0%B0%D0%BC%D1...,Дни памяти и примирения (англ. Time of remembr...
9494,9494,Война интеллекта с глупостью,/wiki/%D0%92%D0%BE%D0%B9%D0%BD%D0%B0_%D0%B8%D0...,Война интеллекта с глупостью — извечная война ...
6233,6233,Леонид (Консулы),/wiki/%D0%9B%D0%B5%D0%BE%D0%BD%D0%B8%D0%B4_(%D...,Леонид — один из ключевых персонажей во вселен...
8963,8963,Бикини-Боттом,/wiki/%D0%91%D0%B8%D0%BA%D0%B8%D0%BD%D0%B8-%D0...,Бикини-Боттом — он же вымышленный город в муль...
4241,4241,Яков Кедми,/wiki/%D0%AF%D0%BA%D0%BE%D0%B2_%D0%9A%D0%B5%D0...,Яков Кедми — израильский государственный деяте...
6988,6988,Среброкрылые Часовые,/wiki/%D0%A1%D1%80%D0%B5%D0%B1%D1%80%D0%BE%D0%...,"Доблестные защитники Ясеневого леса, сражающие..."
